<a href="https://colab.research.google.com/github/Javad-Manashti/yolov8m-pothole-segmentation/blob/main/Bunny_Llama_3_8B_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers accelerate pillow



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!!pip install accelerate


['Requirement already satisfied: accelerate in /usr/local/lib/python3.10/dist-packages (0.29.3)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.10/dist-packages (from accelerate) (1.25.2)',
 'Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.10/dist-packages (from accelerate) (24.0)',
 'Requirement already satisfied: psutil in /usr/local/lib/python3.10/dist-packages (from accelerate) (5.9.5)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/python3.10/dist-packages (from accelerate) (6.0.1)',
 'Requirement already satisfied: torch>=1.10.0 in /usr/local/lib/python3.10/dist-packages (from accelerate) (2.2.1+cu121)',
 'Requirement already satisfied: huggingface-hub in /usr/local/lib/python3.10/dist-packages (from accelerate) (0.20.3)',
 'Requirement already satisfied: safetensors>=0.3.1 in /usr/local/lib/python3.10/dist-packages (from accelerate) (0.4.3)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
from PIL import Image
import warnings
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
from PIL import Image
import warnings
# Clear any cached memory (useful in interactive environments like Jupyter or Colab)
torch.cuda.empty_cache()

# Optional: Set environment variable to manage memory allocation better
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


# Initialize the Accelerator
accelerator = Accelerator(mixed_precision='fp16')

# Load the model and tokenizer
# Note: Depending on the model size, you may still run out of memory here
model = AutoModelForCausalLM.from_pretrained('BAAI/Bunny-Llama-3-8B-V', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('BAAI/Bunny-Llama-3-8B-V', trust_remote_code=True)

# Prepare the model and tokenizer for acceleration
model, tokenizer = accelerator.prepare(model, tokenizer)

# Text prompt
prompt = 'Please analyze the potholes detected. Is it correct? Is that cover all the potholes in the images with the red area?'
text = f"A chat between a user and an artificial intelligence assistant. The assistant works as a professional annotation specialist. USER: <image>\n{prompt} ASSISTANT:"
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = accelerator.prepare(input_ids)

# Load and process image
image_path = '/content/image5-ph.jpg'  # Update the path to where your image is
image = Image.open(image_path)

# Assuming model.process_images() returns a tensor that can be processed by the model
# Note: You will need to check the proper usage of model.process_images() in its documentation
image_tensor = model.process_images([image], model.config)
image_tensor = accelerator.prepare(image_tensor)

# Generate response without manually specifying the device
output_ids = accelerator.unwrap_model(model).generate(
    input_ids,
    images=image_tensor,  # Ensure that images parameter is supported by your model
    max_new_tokens=100,
    use_cache=True
)[0]

# Decoding the output
response = tokenizer.decode(output_ids[len(input_ids[0]):], skip_special_tokens=True)
print(response.strip())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

configuration_bunny_llama.py:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/BAAI/Bunny-Llama-3-8B-V:
- configuration_bunny_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bunny_llama.py:   0%|          | 0.00/125k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/80.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]